# Rapport Final du Projet VRP : Validation de la Pipeline

## 1. Initialisation et Chargement des Modules

Cette section configure l'environnement en important toutes les briques de l'architecture modulaire (`src/`). 
Nous vérifions ici que la séparation des responsabilités (Contrats, Juge, Pilote, Moteur) fonctionne sans erreur.



In [12]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import List, Dict

# Assurez-vous que le chemin du projet est correct pour l'import des modules src
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Importation des fonctions et des structures de données
from src.contracts import Instance, Solution
from src.instance_loader import load_instance
from src.initial_solution import build_clarke_wright_solution
from src.optimization_loop import optimization_loop

print("✅ Pipeline chargée : les modules sont prêts à l'emploi.")

✅ Pipeline chargée : les modules sont prêts à l'emploi.


## 2. Exécution du Solveur (Pipeline complète)

Nous lançons un essai unique sur l'instance de référence **C101** pour démontrer le flux de données de bout en bout :

**Loader** ➔ **Initialiseur** ➔ **Pilote (ALNS)** ➔ **Résultats**

In [19]:
# --- Configuration de la Démo ---
TEST_INSTANCE_NAME = "C101"
MAX_ITER_DEMO = 500  # Budget de recherche
SEED = 42

print(f"--- Lancement de la simulation unique sur : {TEST_INSTANCE_NAME} ---")

# 1. CHARGEMENT (Le Loader crée l'Instance et lit le Coût Optimal)
try:
    # --- CORRECTION DU CHEMIN D'ACCÈS FINAL ---
    # Le chemin remonte au dossier parent (..) pour trouver le dossier 'data'
    instance_path = os.path.join("..", "data", f"{TEST_INSTANCE_NAME}.txt")
    
    # Gérer les majuscules si le fichier est .TXT
    if not os.path.exists(instance_path):
        instance_path = os.path.join("..", "data", f"{TEST_INSTANCE_NAME}.TXT")

    instance, optimal_sol = load_instance(instance_path)
    
    optimal_cost = optimal_sol.cost
    print(f"1. ✅ Chargement de l'Instance : Coût Optimal = {optimal_cost:.2f}")
except Exception as e:
    print(f"1. ❌ ÉCHEC DU CHARGEMENT : {e}")
    raise

# 2. INITIALISATION (L'Initialiseur crée la Solution)
try:
    initial_solution = build_clarke_wright_solution(instance)
    print(f"2. ✅ Solution Initiale (C&W) générée. Coût de départ = {initial_solution.cost:.2f}")
except Exception as e:
    print(f"2. ❌ ÉCHEC DE L'INITIALISATION : {e}")
    raise

# 3. OPTIMISATION (Le Pilote lance l'ALNS)
print(f"3. ⏳ Lancement de l'optimisation ALNS...")
start_time = time.time()
try:
    history = optimization_loop(instance, initial_solution, max_iter=MAX_ITER_DEMO, patience=500, seed=SEED)
except Exception as e:
    print(f"3. ❌ ERREUR FATALE DE L'ALNS : {e}")
    raise
end_time = time.time()

# 4. RÉSULTATS
final_best_cost = history["cost_best"][-1]
gap_pct = ((final_best_cost - optimal_cost) / optimal_cost) * 100

print("\n--- Synthèse du Résultat ---")
print(f"Coût Initial (C&W) : {initial_solution.cost:.2f}")
print(f"Coût Final (ALNS)  : {final_best_cost:.2f}")
print(f"Coût Optimal (Record): {optimal_cost:.2f}")
print(f"Temps d'exécution  : {end_time - start_time:.2f} secondes")
print(f"📈 Gap final vs Optimal : {gap_pct:.2f}%")
print("\n🎉 Le pipeline de bout en bout est validé.")

--- Lancement de la simulation unique sur : C101 ---
1. ❌ ÉCHEC DU CHARGEMENT : [Errno 2] No such file or directory: 'data/..\\data\\C101.txt.txt'


FileNotFoundError: [Errno 2] No such file or directory: 'data/..\\data\\C101.txt.txt'